In [100]:
# Import libraries
import pandas as pd 
import plotly as py
import plotly.express as px
import plotly.graph_objs as go

In [101]:
# Read Data
cov = pd.read_csv("covid_19_data.csv")

In [102]:
# check column data types
cov.dtypes

SNo                 int64
ObservationDate    object
Province/State     object
Country/Region     object
Last Update        object
Confirmed           int64
Deaths              int64
Recovered           int64
dtype: object

In [103]:
# change ObservationDate column to datetime as it wasn't sorting correctly as object
cov['ObservationDate']=cov['ObservationDate'].astype('datetime64[ns]') 

In [104]:
# Rename columns
cov = cov.rename(columns={'Country/Region':'Country','ObservationDate':'Date','Confirmed':'Confirmed Cases'})

In [105]:
# Create new dataframe grouped by country and date sorted by date descending
cov_countries = cov.groupby(['Country', 'Date']).sum().reset_index().sort_values('Date', ascending=False)
# Only keep the most current date with data by dropping duplicate rows and keeping the first records
cov_countries = cov_countries.drop_duplicates(keep='first',subset = ['Country'])
cov_countries

,Country,Date,SNo,Confirmed Cases,Deaths,Recovered
8035,US,2020-04-16,910756,667801,32916,54703
2115,Ecuador,2020-04-16,15497,8225,403,838
4390,Liechtenstein,2020-04-16,15545,79,1,55
2178,Egypt,2020-04-16,15498,2673,196,596
2207,El Salvador,2020-04-16,15499,164,6,33
...,...,...,...,...,...,...
5854,Palestine,2020-03-09,4322,22,0,0
7250,St. Martin,2020-03-09,4412,2,0,0
6226,Republic of Ireland,2020-03-08,4067,21,0,0
0,Azerbaijan,2020-02-28,2664,1,0,0


In [106]:
#Create variable showing latest available date in graph title
latestdate=cov['Date'].max().strftime("%m/%d/%Y")
latestdate

'04/16/2020'

In [107]:
# Create the plotly Choropleth map showing the country data from the latest available date
# Roll over countries for info
fig = go.Figure(data=go.Choropleth(
    #column to use as locations
    locations = cov_countries['Country'],
    #built in plotly mode to use name of countries 
    locationmode = 'country names',
    #use Confirmed Cases columns for values
    z = cov_countries['Confirmed Cases'],
    #text = cov_countries['text'],
    colorscale = 'bluyl',
    marker_line_color = 'black',
    marker_line_width = 1,
))

fig.update_layout(
    #text using the previously set latestdate variable for title
    title_text = 'Confirmed Covid-19 Cases as of' + ' ' + str(latestdate),
    title_x = 0.5,
    geo=dict(
        showframe = False,
        projection_type = 'equirectangular'
    )
)
fig.show()

In [108]:
# Including screenshot gif in case map is not rendered outside of Jupyter

<img src="staticmap.jpg" align='left' width="980" height="500" />

In [109]:
# Copying previous cell and changing the values to show deaths instead of cases
# Create the plotly Choropleth map showing the country data from the latest available date
# Roll over countries for info 
fig = go.Figure(data=go.Choropleth(
    #column to use as locations
    locations = cov_countries['Country'],
    #built in plotly mode to use name of countries 
    locationmode = 'country names',
    #use Confirmed Cases columns for values
    z = cov_countries['Deaths'],
    colorscale = 'bluered',
    marker_line_color = 'black',
    marker_line_width = 1,
))

fig.update_layout(
     #text using the previously set latestdate variable for title
    title_text = 'Confirmed Covid-19 Deaths as of' + ' ' + str(latestdate),
    title_x = 0.5,
    geo=dict(
        showframe = False,
        projection_type = 'equirectangular'
    )
)

In [110]:
# Including screenshot gif in case map is not rendered outside of Jupyter

<img src="staticmap_deaths.jpg" align='left' width="980" height="600" />

In [111]:
# Modify original dataframe to group by date and country for showing all dates
cov_countrydate = cov.groupby(['Date','Country']).sum().reset_index()
cov_countrydate

,Date,Country,SNo,Confirmed Cases,Deaths,Recovered
0,2020-01-22,Hong Kong,13,0,0,0
1,2020-01-22,Japan,36,2,0,0
2,2020-01-22,Macau,21,1,0,0
3,2020-01-22,Mainland China,535,547,17,28
4,2020-01-22,South Korea,38,1,0,0
...,...,...,...,...,...,...
8478,2020-04-16,West Bank and Gaza,15626,374,2,63
8479,2020-04-16,Western Sahara,15627,6,0,0
8480,2020-04-16,Yemen,15628,1,0,0
8481,2020-04-16,Zambia,15629,48,2,30


In [112]:
#converting date back to string at this point as it's needed for the animated date-changing map to follow
cov_countrydate['Date']=cov_countrydate['Date'].astype(str) 

In [113]:
#Create variable showing earliest available date in graph title
earliestdate=cov['Date'].min().strftime("%m/%d/%Y")
earliestdate

'01/22/2020'

In [116]:
# Creating the animated choropleth map to show changes over time
# Roll over countries for info after playing
fig = px.choropleth(cov_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed Cases", 
                    hover_name="Country",
                    color_continuous_scale="bluyl",
                    #select date column for animating
                    animation_frame="Date"
                   )
fig.update_layout(
    title_text = 'Global Spread of Covid-19 Confirmed Cases from ' + str(earliestdate) + ' to ' + str(latestdate),
    title_x = 0.5,
    geo=dict(
        showframe = False,
    ))
    
fig.show()

In [117]:
#including autoplay recorded animated gif

<img src="animated_map.gif" align='left' width="980" height="600" />

In [118]:
# Changing previous animated map to show deaths instead of cases 
# Creating the animated choropleth map to show changes over time
# Roll over countries for info after playing
fig = px.choropleth(cov_countrydate, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Deaths", 
                    hover_name="Country",
                    color_continuous_scale="bluered",
                    #select date column for animating
                    animation_frame="Date"
                   )
fig.update_layout(
    title_text = 'Global Spread of Covid-19 Confirmed Deaths from ' + str(earliestdate) + ' to ' + str(latestdate),
    title_x = 0.5,
    geo=dict(
        showframe = False,
    ))
    
fig.show()

In [119]:
#including autoplay recorded animated gif

<img src="animated_map_deaths.gif" align='left' width="980" height="600" />